In [1]:
import numpy as np
import time
import concurrent.futures as cf

In [4]:
#@np.vectorize
def split_particles(position, charge, mins, maxs, sizes):
    '''
    positions = array of size (Np, N_dim)
    mins  = (xmin, ymin, zmin)
    maxs  = (xmax, ymax, zmax)
    sizes = (Nx, Ny, Nz)
    '''

    if ((mins.shape != maxs.shape) | (mins.shape != sizes.shape)):
        print('Dimension mismatch between mins, maxs, and sizes!!')
        return None
    
    if (np.any( position - mins < 0 ) | np.any( position - maxs >= 0 )):
        print(np.any( position - mins < 0 ) )
        print(np.any( position - maxs >= 0 ) )
        print('Particle position falls outside the boundary specified!')
        return None
    
    deltas = (maxs - mins) / (sizes - 1)
        #print('deltas: ',deltas)
    floors = np.floor( (position - mins)/deltas + 1 )  # index of the nearest "floor point"
    floors = floors.astype(int)
        #print('floors: ', floors)
    weights = ((mins - position) + floors*deltas) / deltas  # werights towards the floor point
        #print('weights: ', weights)
    
    dim = sizes.shape[0]
    if (dim > 3) :
        print('Dimension > 3 detected!!')
        return None

    elif (dim == 3):

        ip = floors[:,0]
        jp = floors[:,1]
        kp = floors[:,2]
        
        w1 = weights[:,0]
        w2 = weights[:,1]
        w3 = weights[:,2]        
 
        t1 = w1*w2*w3*charge
        t2 = w1*(1-w2)*w3*charge
        t3 = w1*(1-w2)*(1-w3)*charge
        t4 = w1*w2*(1-w3)*charge

        t5 = (1-w1)*w2*w3*charge
        t6 = (1-w1)*(1-w2)*w3*charge
        t7 = (1-w1)*(1-w2)*(1-w3)*charge
        t8 = (1-w1)*w2*(1-w3)*charge

        indexes = np.array([ip,jp,kp])
        contrib = np.array([t1,t2,t3,t4,t5,t6,t7,t8])
        
    elif (dim == 2):
        ip = floors[:,0]
        jp = floors[:,1]
        
        w1 = weights[:,0]
        w2 = weights[:,1]     
 
        t1 = w1*w2*charge
        t2 = w1*(1-w2)*charge
        t3 = (1-w1)*(1-w2)*charge
        t4 = (1-w1)*w2*charge

        indexes = np.array([ip,jp])
        contrib = np.array([t1,t2,t3,t4])
    
    elif (dim == 1):

        ip = floors[:,0]    
        w1 = weights[:,0]   
 
        t1 = w1*charge
        t2 = (1-w1)*charge

        indexes = np.array([ip])
        contrib = np.array([t1,t2])
        
    return indexes, contrib

In [9]:
def deposit_particles( sizes, indexes, contrib):
    """
    Deposit the "splitted particles" on an empty grid.
    Use this function after the "split_particles" function.
    For large Np this function can be slow.
    """
    
    Np = indexes.shape[1]
    charge_grid = np.zeros(sizes)
    # Populate charge_grid
    for n in range(Np):
        (ip, jp) = indexes[:,n] # depositting index of the nth particle
        (t1, t2, t3, t4) = contrib[:,n] # contribtuion of the nth particle
    
        charge_grid[ip-1][jp-1] = charge_grid[ip-1][jp-1] + t1
        charge_grid[ip-1][jp]   = charge_grid[ip-1][jp]   + t2

        charge_grid[ip][jp]     = charge_grid[ip][jp]     + t3
        charge_grid[ip][jp-1]   = charge_grid[ip][jp-1]   + t4
        
    return charge_grid

def deposit_particles2( indexes, contrib):
    """
    Deposit the "splitted particles" on an empty grid.
    Use this function after the "split_particles" function.
    For large Np this function can be slow.
    """
    
    Np = indexes.shape[0]
    #charge_grid = np.zeros(sizes)
    
    charge_grid = np.zeros(np.array([105,101]))
    
    # Populate charge_grid
    for n in range(Np):
        (ip, jp) = indexes[n,:] # depositting index of the nth particle
        (t1, t2, t3, t4) = contrib[n,:] # contribtuion of the nth particle
    
        charge_grid[ip-1][jp-1] = charge_grid[ip-1][jp-1] + t1
        charge_grid[ip-1][jp]   = charge_grid[ip-1][jp]   + t2

        charge_grid[ip][jp]     = charge_grid[ip][jp]     + t3
        charge_grid[ip][jp-1]   = charge_grid[ip][jp-1]   + t4
        
    return charge_grid

# Testing

In [7]:
np.random.seed(0)

In [17]:
charge = 25
mins = np.array([-0.2,-0.2,-0.2])  # Lower bounds of the grid
maxs = np.array([0.2,0.2,0.2])     # Upper bounds of the grid
sizes = np.array([500,500,500])

#beam1 = np.array([[1,1,1]])
#beam2 = np.array([[0.1,0.1,0.1]])
#beam3 = np.array([[1,1,1],[0.1,0.1,0.1],[2.3,2.5,2.9]])

beam4 = np.random.normal(0, 0.02, (100000, 3))

Np = beam4.shape[0]




In [7]:
Np = 1000000

mins = np.array([-51E-6,-51E-6])  # Lower bounds of the grid
maxs = np.array([51E-6,51E-6])     # Upper bounds of the grid
sizes = np.array([105, 101])

Q = 160E-12
charges = np.ones(Np)*Q/Np

beam = np.random.normal(0, 10E-6 , size=(6, Np))
(x_b, xp_b, y_b, yp_b, z_b, zp_b) = beam
zx_positions = np.stack((z_b, x_b)).T

In [9]:
#indexes, contrib = split_particles(beam4, charge, mins, maxs, sizes)

In [107]:
sizes

array([105, 101])

In [8]:
indexes, contrib = split_particles(zx_positions, charges, mins, maxs, sizes)
t1 = time.time();
charge_grid = deposit_particles(sizes, indexes, contrib)
t2 = time.time();
print('Depositting particles takes:', t2 - t1, 's')

True
False
Particle position falls outside the boundary specified!


TypeError: cannot unpack non-iterable NoneType object

In [268]:
indexesT = indexes.T
contribT = contrib.T

In [267]:
t1 = time.time();
with cf.ProcessPoolExecutor(max_workers = 12) as executor:
    #charge_grid2 = deposit_particles2(sizes, indexesT, contribT)
    temp = executor.map( deposit_particles2, indexesT, contribT)
    #print(list(temp))
    charge_grid2 = np.array(list(temp))
    
t2 = time.time();
print('Depositting particles takes:', t2 - t1, 's')

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [277]:
t1 = time.time()
charge_grid2 = deposit_particles2( indexesT, contribT)
t2 = time.time()
print(t2-t1)

8.297629117965698


In [273]:
charge_grid2[50][50]

2.5304964795983684e-13

In [285]:
iii = np.array(np.array_split(indexesT,20))
jjj = np.array(np.array_split(contribT,20))
print(iii.shape)
print(jjj.shape)

(20, 50000, 2)
(20, 50000, 4)


In [281]:
t1 = time.time()
temp = map(deposit_particles2, iii, jjj)
gather = list(temp)
t2 = time.time()
print(t2-t1)

8.093605756759644


In [286]:
t1 = time.time()
with cf.ProcessPoolExecutor(max_workers = 20) as executor:
    temp = executor.map(deposit_particles2, iii, jjj, chunksize = 1)
    gather2 = np.array(list(temp))
t2 = time.time()
print(t2-t1)

2.4571573734283447


In [265]:
gather2.shape

(5, 105, 101)

In [284]:
gg =  gather[0] + gather[1] + gather[2] + gather[3] + gather[4] 

In [275]:
gg[50][50]

2.5304964795983603e-13

In [276]:
np.allclose(gg,charge_grid2)

True

In [124]:
charge_grid = deposit_particles2(sizes, indexesT, contribT)


In [193]:
iii = np.array(np.array_split(indexesT,5))
jjj = np.array(np.array_split(contribT,5))
jjj.shape


(5, 200000, 4)

In [224]:
cg1 = deposit_particles2(sizes, indexesT[0:10000], contribT[0:10000])
cg2 = deposit_particles2(sizes, indexesT[10000:20000], contribT[10000:20000])
cg3 = deposit_particles2(sizes, indexesT[20000:30000], contribT[20000:30000])
cg4 = deposit_particles2(sizes, indexesT[30000:40000], contribT[30000:40000])
cg5 = deposit_particles2(sizes, indexesT[40000:50000], contribT[40000:50000])

In [225]:
cg = cg1 + cg2 +cg3 +cg4 +cg5

In [163]:
charge_grid_saved[30][33]

6.200954313836813e-15

In [106]:
charge_grid_saved[50][50]

2.4007138414907846e-13

In [109]:
charge_grid_saved[50][51]

2.5487301874247584e-13

In [110]:
charge_grid_saved[51][50]

2.614049555215493e-13

In [ ]:
def dep_one(sizes, indexes, contrib):
    """
    Deposit the "splitted particles" on an empty grid.
    Use this function after the "split_particles" function.
    For large Np this function can be slow.
    """
    
    charge_grid = np.zeros(sizes)
    # Populate charge_grid

    (ip, jp) = indexes # depositting index of the nth particle
    (t1, t2, t3, t4) = contrib # contribtuion of the nth particle
    
    charge_grid[ip-1][jp-1] = charge_grid[ip-1][jp-1] + t1
    charge_grid[ip-1][jp]   = charge_grid[ip-1][jp]   + t2

    charge_grid[ip][jp-1]   = charge_grid[ip][jp-1]   + t3
    charge_grid[ip][jp]     = charge_grid[ip][jp]     + t4
        
    return charge_grid

In [51]:
indexesT = indexes.T
contribT = contrib.T

In [57]:
contribT[0]

array([2.10857803e-15, 3.96094205e-16, 1.09960614e-16, 5.85367148e-16])

In [118]:
from csr2d.core import psi_s, psi_x
zmin = -0.001
xmin = -0.001
zmax = 0.001
xmax = 0.001
Nz = 105
Nx = 101

zvec2 = np.linspace(2*zmin,2*zmax,2*Nz)
xvec2 = np.linspace(2*xmin,2*xmax,2*Nx)
zm2, xm2 = np.meshgrid(zvec2, xvec2, indexing='ij')
    #psi_s_grid = psi_s(zm2,xm2,beta)

    
beta = 0.999
#beta*np.ones(2*Nz)
beta_grid = beta*np.ones(zm2.shape)

t1 = time.time()
with cf.ProcessPoolExecutor(max_workers = 12) as executor:
    temp = executor.map(psi_s, zm2, xm2, beta_grid, chunksize=12)
    psi_s_grid = np.array(list(temp))
t2 = time.time()

print(t2-t1)
    

0.1355431079864502


In [100]:
psi_s_grid

array([[-0.50153527, -0.49654269, -0.49155017, ...,  0.48832118,
         0.49327184,  0.49822228],
       [-0.50636707, -0.5013264 , -0.49628579, ...,  0.49304825,
         0.49804674,  0.50304501],
       [-0.51129313, -0.50620343, -0.50111379, ...,  0.49786745,
         0.5029147 ,  0.50796174],
       ...,
       [-8.92166038, -8.92012894, -8.9185954 , ..., -8.58320182,
        -8.58136807, -8.57953334],
       [-8.89208422, -8.89056391, -8.88904153, ..., -8.55645466,
        -8.5546374 , -8.55281919],
       [-8.86288795, -8.86137859, -8.85986721, ..., -8.53003795,
        -8.52823688, -8.52643487]])

In [98]:
psi_s_grid

array([[-0.50153527, -0.49654269, -0.49155017, ...,  0.48832118,
         0.49327184,  0.49822228],
       [-0.50636707, -0.5013264 , -0.49628579, ...,  0.49304825,
         0.49804674,  0.50304501],
       [-0.51129313, -0.50620343, -0.50111379, ...,  0.49786745,
         0.5029147 ,  0.50796174],
       ...,
       [-8.92166038, -8.92012894, -8.9185954 , ..., -8.58320182,
        -8.58136807, -8.57953334],
       [-8.89208422, -8.89056391, -8.88904153, ..., -8.55645466,
        -8.5546374 , -8.55281919],
       [-8.86288795, -8.86137859, -8.85986721, ..., -8.53003795,
        -8.52823688, -8.52643487]])

In [72]:
def csum(vec, shift):
    return vec[0]+vec[1]+vec[2] + shift

In [236]:
kaka.shape

(2, 4)

In [75]:
kaka = np.array([[1,2,3,4],[5,6,7,8]])
mm = map(csum,kaka,np.ones(2))
list(mm)

[7.0, 19.0]

In [81]:

with cf.ProcessPoolExecutor(max_workers = 12) as executor:
    temp = executor.map(csum,kaka,np.ones(1))
    result = np.array(list(temp))

In [82]:
result

array([7.])

# Scratch below

In [ ]:
tt1 = time.time();
# Method 1

charge_grid = np.zeros(sizes)
# Populate charge_grid
for n in range(Np):
    (ip, jp, kp) = indexes[:,n] # depositting index of the nth particle
    (t1, t2, t3, t4, t5, t6, t7, t8) = contrib[:,n] # contribtuion of the nth particle
    
    charge_grid[ip-1][jp-1][kp-1] = charge_grid[ip-1][jp-1][kp-1] + t1
    charge_grid[ip-1][jp][kp-1]   = charge_grid[ip-1][jp][kp-1]   + t2
    charge_grid[ip-1][jp][kp]     = charge_grid[ip-1][jp][kp]     + t3
    charge_grid[ip-1][jp-1][kp]   = charge_grid[ip-1][jp-1][kp]   + t4

    charge_grid[ip][jp-1][kp-1]   = charge_grid[ip][jp-1][kp-1] + t5
    charge_grid[ip][jp][kp-1]     = charge_grid[ip][jp][kp-1]   + t6
    charge_grid[ip][jp][kp]       = charge_grid[ip][jp][kp]     + t7
    charge_grid[ip][jp-1][kp]     = charge_grid[ip][jp-1][kp]   + t8

#charge_grid[50][56][50];

tt2 = time.time();

print('time:', tt2 - tt1)

In [ ]:
charge_grid[250][250][250]

In [ ]:
charge_grid[250:254,250:254,250:254]

In [11]:
y = np.random.random((500, 500, 500))
y.shape

(500, 500, 500)

In [12]:
# this saves memory
def hist_grid(sizes,indexes,contrib):
    
    xedges = np.arange(sizes[0]+1)
    yedges = np.arange(sizes[1]+1)
    zedges = np.arange(sizes[2]+1)

    h1 = np.sum( [ 
     np.histogramdd(indexes.T, bins = (xedges+1,yedges+1,zedges+1), weights=contrib[0])[0],
     np.histogramdd(indexes.T, bins = (xedges+1,yedges,zedges+1), weights=contrib[1])[0],
     np.histogramdd(indexes.T, bins = (xedges+1,yedges,zedges), weights=contrib[2])[0],
     np.histogramdd(indexes.T, bins = (xedges+1,yedges+1,zedges), weights=contrib[3])[0],

     np.histogramdd(indexes.T, bins = (xedges,yedges+1,zedges+1), weights=contrib[4])[0],
     np.histogramdd(indexes.T, bins = (xedges,yedges,zedges+1), weights=contrib[5])[0],
     np.histogramdd(indexes.T, bins = (xedges,yedges,zedges), weights=contrib[6])[0],
     np.histogramdd(indexes.T, bins = (xedges,yedges+1,zedges), weights=contrib[7])[0]])

    return h1


In [13]:
tt1 = time.time();
hist_grid(sizes,indexes,contrib)

tt2 = time.time();
print('time:', tt2 - tt1)

time: 21.767561674118042


In [6]:
tt1 = time.time();

xedges = np.arange(sizes[0]+1)
yedges = np.arange(sizes[1]+1)
zedges = np.arange(sizes[2]+1)

h1 = np.histogramdd(indexes.T, bins = (xedges+1,yedges+1,zedges+1), weights=contrib[0])[0]
h2 = np.histogramdd(indexes.T, bins = (xedges+1,yedges,zedges+1), weights=contrib[1])[0]
h3 = np.histogramdd(indexes.T, bins = (xedges+1,yedges,zedges), weights=contrib[2])[0]
h4 = np.histogramdd(indexes.T, bins = (xedges+1,yedges+1,zedges), weights=contrib[3])[0]

h5 = np.histogramdd(indexes.T, bins = (xedges,yedges+1,zedges+1), weights=contrib[4])[0]
h6 = np.histogramdd(indexes.T, bins = (xedges,yedges,zedges+1), weights=contrib[5])[0]
h7 = np.histogramdd(indexes.T, bins = (xedges,yedges,zedges), weights=contrib[6])[0]
h8 = np.histogramdd(indexes.T, bins = (xedges,yedges+1,zedges), weights=contrib[7])[0]

htot = h1+h2+h3+h4+h5+h6+h7+h8
htot[50][56][50];

tt2 = time.time();
print('time:', tt2 - tt1)

time: 11.502238750457764


In [ ]:
import concurrent.futures as cf

In [ ]:
# parallel?
with concurrent.futures.ProcessPoolExecutor(max_workers = 24) as executor:
    temp = executor.map(psi_s, zm2, xm2, beta_grid)
    result2 = np.array(list(temp))

In [ ]:
htot[250:254,250:254,250:254]

In [ ]:
htot[48:52,48:52,48:52]

In [ ]:
r = np.ones((4,3))
np.histogramdd(r, bins = (3,3,3), weights=(0.1,0.3,0.5,0.7))

In [ ]:
xedges = [0, 1,2,3,4,5]
yedges = [0, 1,2,3,4,5]
np.histogram2d([1,1,2,3,4,4],[1,2,2,3,4,4],bins = (xedges, yedges), weights=[1,2,3,4,5,6])

In [ ]:
np.histogram([1, 2, 1], bins=[0, 1, 2, 3])

In [ ]:
np.histogram([[1, 2, 1], [1, 0, 1]], bins=[0,1,2,3])

In [ ]:
np.histogram([1,2,3,4,5,6,7,7,7], bins=7)